In [ ]:
#initializing the spark shell with hudi packages
./spark-shell --packages org.apache.hudi:hudi-spark-bundle_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.4
    --conf 'spark.serializer=org.apache.spark.serializer.KryoSerializer'

In [ ]:
#importing hudi libraries in Scala
import org.apache.hudi.QuickstartUtils._
import scala.collection.JavaConversions._
import org.apache.spark.sql.SaveMode._
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.config.HoodieWriteConfig._

In [ ]:
#table name
val tableName = "hudi_trips_cow"

#inside spark2: a path for saving the partitions
val basePath = "file:///tmp/hudi_trips_cow"

#a fancy data generator for random data insertion
val dataGen = new DataGenerator

In [ ]:
#generating new trips
val inserts = convertToStringList(dataGen.generateInserts(10))

#loading them into a dataframe
val df = spark.read.json(spark.sparkContext.parallelize(inserts, 2))

#writing the dataframe into a Hudi table
df.write.format("hudi").
  options(getQuickstartWriteConfigs).
  option(PRECOMBINE_FIELD_OPT_KEY, "ts").
  option(RECORDKEY_FIELD_OPT_KEY, "uuid").
  option(PARTITIONPATH_FIELD_OPT_KEY, "partitionpath").
  option(TABLE_NAME, tableName).
  mode(Overwrite).
  save(basePath)

In [ ]:
#loading into a dataframe for query purpose
val tripsSnapshotDF = spark.
  read.
  format("hudi").
  load(basePath + "/*/*/*/*").
  createOrReplaceTempView("hudi_trips_snapshot")


In [ ]:
#querying the table
spark.sql("select fare, begin_lon, begin_lat, ts from  hudi_trips_snapshot where fare > 20.0").show()